In [1]:
!pip install pandas numpy scikit-learn openai

In [3]:
import pandas as pd

content_df = pd.read_csv("/content/Content Recommendations-Grid view.csv")
user_df = pd.read_csv("/content/User_Content_Preferences - Form Responses 1.csv")

content_df.head(), user_df.head()


(   Content_ID                                   Title  \
 0           1    Artificial Intelligence in Marketing   
 1           2  Data Analytics for Marketing Campaigns   
 2           3  Personalization in Digital Advertising   
 3           4                   AdTech Trends in 2025   
 4           5     AI-Powered Ad Campaign Optimization   
 
                                          Description  \
 0  This article explains how artificial intellige...   
 1  A detailed guide on how data analytics helps m...   
 2  This content explains how personalized adverti...   
 3  An overview of the latest AdTech trends includ...   
 4  A podcast episode featuring industry experts d...   
 
                                                  URL     Category Platform  
 0            https://www.ibm.com/topics/ai-marketing           AI     Blog  
 1  https://www.salesforce.com/resources/articles/...    Marketing     Blog  
 2  https://www.mckinsey.com/capabilities/growth-m...  Advertising     B

In [4]:
from openai import OpenAI
client = OpenAI("API_KEY")

In [5]:
def get_embedding(text):
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return response.data[0].embedding

In [6]:
content_df["embedding"] = content_df["Description"].apply(get_embedding)


In [8]:
print(user_df.columns.tolist())

['Timestamp', 'Name ', 'Which topics are you interested in?  ', 'What type of content do you prefer?  ']


In [10]:
user_interest = user_df.iloc[0]["Which topics are you interested in?  "]
user_embedding = get_embedding(user_interest)

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

content_embeddings = np.vstack(content_df["embedding"].values)
similarities = cosine_similarity([user_embedding], content_embeddings)[0]

content_df["similarity"] = similarities
recommendations = content_df.sort_values("similarity", ascending=False).head(3)

recommendations[["Title", "URL"]]


,Title,URL
0,Artificial Intelligence in Marketing,https://www.ibm.com/topics/ai-marketing
6,AI in Social Media Marketing,https://sproutsocial.com/insights/ai-social-me...
9,AI for Customer Engagement,https://www.oracle.com/customer-experience/wha...
